In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from boxes import BoxCollection
from polygonal import build_min_distance

# Offline preprocessing

In [ ]:
from time import time

d = 2
n = 50
sides = [2, .5]

B = BoxCollection.generate_grid(d, n, sides, seed=0)
G = B.line_graph('optimize')

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
# G.plot()

# Polygonal pahse: shortest-path problem

In [ ]:
start = np.zeros(2)
goal = np.ones(2) * (n - 1)

In [ ]:
# Solves all-to-one shortest path for a fixed goal location.
discrete_planner, runtime = G.shortest_path(goal)
runtime

In [ ]:
# Given start location finds box sequence (and initial trajectory).
box_seq, length, runtime = discrete_planner(start)
length, runtime

# Polygonal pahse: iterative shortening

In [ ]:
# Builds optimization problem (can be done offline).
max_boxes = len(box_seq)
continuous_planner = build_min_distance(B, max_boxes, solver='ECOS')

In [ ]:
traj, length, solver_time, runtime = continuous_planner(box_seq, start, goal)
length, solver_time, runtime

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
plt.plot(*traj.T, c='b')

In [ ]:
# Builds optimization problem (can be done offline).
iterative = True
max_boxes = 2 * n
continuous_planner = build_min_distance(B, max_boxes, iterative, solver='ECOS')

# Refresh discrete plan.
box_seq, runtime, length = discrete_planner(start)

In [ ]:
box_seq, traj, length, solver_time, runtime = continuous_planner(box_seq, start, goal)
length, solver_time, runtime

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
B.plot2d(subset=box_seq, fc='None', ec='r')
plt.plot(*traj.T, c='b', marker='+')

# Smooth phase

In [ ]:
import cvxpy as cp
from bezier import optimize_bezier, optimize_bezier_with_retiming

l = np.array([B.boxes[i].l for i in box_seq])
u = np.array([B.boxes[i].u for i in box_seq])

lam = {3: 1}
initial = {0: start}
final = {0: goal}

T = n
h0 = np.linalg.norm(traj[1:] - traj[:-1], axis=1)
h0 *= T / sum(h0)

In [ ]:
solver = 'ECOS'
bez0, cost0, runtime0 = optimize_bezier(l, u, h0, lam, initial, final, solver=solver)[:-1]
cost0, runtime0

In [ ]:
bez, cost, h, runtime = optimize_bezier_with_retiming(l, u, h0, lam, initial, final,
                                                      verbose=True, solver=solver)

In [ ]:
plt.figure(figsize=(5, 5))
B.plot2d(alpha=.5)
bez0.plot2d(c='red')
bez.plot2d(c='blue')

In [ ]:
from itertools import accumulate

plt.figure()
times0 = [0] + list(accumulate(h0))
times = [0] + list(accumulate(h))
segments = np.arange(len(h) + 1)
plt.plot(times0, segments, label='Initial trajectory')
plt.plot(times, segments, label='Final trajectory')
plt.xlabel('Time')
plt.ylabel('Trajectory segments')
plt.grid()
plt.xlim([0, T])
plt.ylim([0, len(h)])
plt.legend()